# Import Moduals

In [1]:
import numpy as np
import pandas as pd
import NeuralNetwork

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.initializers import GlorotUniform
from keras.metrics import CategoricalAccuracy
from keras.losses import MeanSquaredError

from importlib import reload

metrics = CategoricalAccuracy()

2023-04-30 23:08:01.818946: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 23:08:02.653866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-30 23:08:03.506543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-30 23:08:03.603154: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

# Preprocessing

In [3]:
# import dataset
from keras.datasets import mnist
# load dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
input_size
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255
# network parameters
batch_size = 128
hidden_units = 128

# Keras' Model

In [76]:
model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size, use_bias=False))
#model.add(Dropout(0.45))
model.add(Dense(hidden_units, use_bias=False))
#model.add(Dropout(0.45))
model.add(Dense(num_labels, use_bias=False))
model.compile(loss=MeanSquaredError(), 
              optimizer=SGD(learning_rate=0.01),
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1005 - accuracy: 0.5426
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0630 - accuracy: 0.7344
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0560 - accuracy: 0.7764
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0526 - accuracy: 0.7976
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0505 - accuracy: 0.8082
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0490 - accuracy: 0.8164
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0479 - accuracy: 0.8219
Epoch 8/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0470 - accuracy: 0.8262
Epoch 9/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0464 - accuracy: 0.8293
Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0458 - accuracy: 0.8317

In [77]:
y_hat = model.predict(x_test)
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

313/313 [==============================] - 0s 936us/step


0.84243333

# My Model

In [7]:
reload(NeuralNetwork)
from NeuralNetwork import MyModel, MyLayer, dropout

m = MyModel()
m.addLayer(MyLayer(input_size, hidden_units))
#m.addLayer(dropout(0.45))
m.addLayer(MyLayer(hidden_units, hidden_units))
#m.addLayer(dropout(0.45))
m.addLayer(MyLayer(hidden_units, num_labels))


m.fit(x_train, y_train, 0.01, epoches = 10, batch_size = batch_size)

Epoch  1: 469/469; Accuracy: 0.4870; Loss: 0.0763: 100%|
Epoch  2: 469/469; Accuracy: 0.7155; Loss: 0.0651: 100%|
Epoch  3: 469/469; Accuracy: 0.7657; Loss: 0.0599: 100%|
Epoch  4: 469/469; Accuracy: 0.7885; Loss: 0.0569: 100%|
Epoch  5: 469/469; Accuracy: 0.8001; Loss: 0.0547: 100%|
Epoch  6: 469/469; Accuracy: 0.8081; Loss: 0.0531: 100%|
Epoch  7: 469/469; Accuracy: 0.8142; Loss: 0.0519: 100%|
Epoch  8: 469/469; Accuracy: 0.8190; Loss: 0.0509: 100%|
Epoch  9: 469/469; Accuracy: 0.8225; Loss: 0.0501: 100%|
Epoch 10: 469/469; Accuracy: 0.8251; Loss: 0.0493: 100%|


In [79]:
y_hat = m.predict(x_test)
metrics.reset_state()
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

0.839

In [4]:
reload(NeuralNetwork)
from NeuralNetwork import MyModel, MyLayer, dropoutS

m = MyModel()
m.addLayer(MyLayer(input_size, 128))
m.addLayer(dropoutS(2))
m.addLayer(MyLayer(128, num_labels))


m.fitS(x_train, y_train, 0.01, epoches = 10, batch_size = batch_size)

Epoch  1: 469/469; Accuracy: 0.1977; Loss: 0.1817: 100%|
Epoch  2: 469/469; Accuracy: 0.3207; Loss: 0.1394: 100%|
Epoch  3: 469/469; Accuracy: 0.3914; Loss: 0.1337: 100%|
Epoch  4: 469/469; Accuracy: 0.4399; Loss: 0.1277: 100%|
Epoch  5: 469/469; Accuracy: 0.4825; Loss: 0.1076: 100%|
Epoch  6: 469/469; Accuracy: 0.5155; Loss: 0.1034: 100%|
Epoch  7: 469/469; Accuracy: 0.5387; Loss: 0.1074: 100%|
Epoch  8: 469/469; Accuracy: 0.5618; Loss: 0.0990: 100%|
Epoch  9: 469/469; Accuracy: 0.5785; Loss: 0.0885: 100%|
Epoch 10: 469/469; Accuracy: 0.5982; Loss: 0.0892: 100%|


In [5]:
y_hat = m.predict(x_test)
metrics.reset_state()
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

0.8207